In [1]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import pylab as pl

import os
import sys

import numpy as np
from scipy import ndimage as ni
import maxflow

from tifffile import imread, imsave
import cPickle as pickle

from skimage.filters import gaussian_filter

from spimagine import volshow

%reload_ext autoreload
%autoreload 2

%matplotlib qt

# %matplotlib inline
# %pylab inline
# pylab.rcParams['figure.figsize'] = (5, 5)

<pyopencl.Device 'HD Graphics 4000' on 'Apple' at 0x1024400>


/Users/jug/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/pyopencl/__init__.py:207: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  "to see more.", CompilerWarning)


could not open /Users/jug/.spimagine
<pyopencl.Device 'HD Graphics 4000' on 'Apple' at 0x1024400>


### Load sampled unit sphere and set parameters

### Loading image to be segmented

In [26]:
# filename = "sample3d_cup1.tif"
filename = '/Volumes/FastData/ProjectGoldstein/Maddox_LITE/mneongreen_membrane_488_90percentpower_nobinning_60xWI_5msexposure_072916_single_timepoint_crop.tif'

sigma = [1,3,3]
image = imread(filename)
image = image - np.mean(image)
image = image / np.var(image)
image_smooth = gaussian_filter(image,sigma)
print 'Image dimensions: ', image.shape
print 'Image min/max:    ', np.min(image), np.max(image)

Image dimensions:  (41, 457, 596)
Image min/max:     -0.314256730763 0.661945663949


In [27]:
volwin = volshow(image, stackUnits = [1.,1.,4.], raise_window=False)
# volwin = volshow(image_smooth, stackUnits = [1.,1.,4.], raise_window=False)

In [54]:
plt.rcParams['figure.figsize'] = (5, 5)

# load pickeled unit sphere sampling
with open('sphere_sampling.pkl','r') as f:
    dictSphereData = pickle.load(f)
    
# sampling parameters
num_columns = dictSphereData['n']
col_vectors = dictSphereData['points']
neighbors = dictSphereData['neighbors']
neighbors_of = dictSphereData['neighbors_of']
num_neighbors = len(neighbors)/num_columns
K = 30       # as in Wu & Chen 2002

# net surface related parameters
max_delta_k = 5

# positioning of net in image
center_x = image.shape[2] / 2
center_y = image.shape[1] / 2
center_z = image.shape[0] / 2
max_r_x = image.shape[2] / 2
max_r_y = image.shape[1] / 2
max_r_z = image.shape[0] / 2
min_r_x = image.shape[2] / 4
min_r_y = image.shape[1] / 4
min_r_z = image.shape[0] / 4

# Building Graph

### Function definitions

In [55]:
from myfunctions import *

In [56]:
# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 
# I LOCALLY OVERWRITE THE FUNCTION FROM myfunctions.py HERE TO PLAY WITH IT
# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 

def build_flow_network( num_columns, neighbors_of, K, num_neighbors, max_delta_k, w_tilde, alpha=0 ):
    '''
    Builds the flow network that can solve the V-Weight Net Surface Problem
    Returns a tuple (g, nodes) consisting of the flow network g, and its nodes.
    '''
    INF = 10000000

    num_nodes = num_columns*K
    num_edges = (num_nodes*num_neighbors*(max_delta_k+max_delta_k+1))/2

    g = maxflow.Graph[float](num_nodes,num_edges)
    nodes = g.add_nodes(num_nodes)

    for i in range(num_columns):

        # connect column to s,t
        for k in range(K):
            if w_tilde[i,k] < 0:
                g.add_tedge(i*K+k, -w_tilde[i,k], 0)
            else:
                g.add_tedge(i*K+k, 0, w_tilde[i,k])

        # connect column to i-chain
        for k in range(1,K):
            g.add_edge(i*K+k, i*K+k-1, INF, 0)

        # connect column to neighbors
        for k in range(K):
            for j in neighbors_of[i]:
                k2 = max(0,k-max_delta_k)
                g.add_edge(i*K+k, j*K+k2, INF, 0)
                # add constant cost penalty \alpha
                g.add_edge(i*K+k, j*K+k, alpha, 0)

    return g, nodes

### Compute weights *w*

In [57]:
center = ( center_x, center_y, center_z)
max_radii = ( max_r_x, max_r_y, max_r_z )
min_radii = ( min_r_x, min_r_y, min_r_z )
w, w_tilde = compute_weights( image_smooth, center, K, max_radii, col_vectors, inverse_order=True, min_radii=min_radii )

### Build and solve flow network

In [58]:
g,nodes = build_flow_network( num_columns, neighbors_of, K, num_neighbors, max_delta_k, w_tilde, 0 )
maxval = g.maxflow()

### Visualize results

In [59]:
size_s_comp = 0
size_t_comp = 0
for n in nodes:
    seg = g.get_segment(n)
    if seg == 0:
        size_s_comp += 1
    else:
        size_t_comp += 1
    #print "Segment of the node ",n,":", seg
print "Size of s and t component: ", size_s_comp, size_t_comp

Size of s and t component:  13638 1722


In [60]:
# imgout = np.zeros(image.shape)
# valtoplace = 1
imgout = np.copy(image)
dotval_in_value = 1.1 * np.max(image)
dotval_out_value = 0.7 * np.max(image) #0.9 * np.min(image)
for i in range(num_columns):
    for k in range(K):
        x = int(center_x + col_vectors[i,0] * min_r_x + col_vectors[i,0] * (K-1-k)/float(K) * (max_r_x-min_r_x))
        y = int(center_y + col_vectors[i,1] * min_r_y + col_vectors[i,1] * (K-1-k)/float(K) * (max_r_y-min_r_y))
        z = int(center_z + col_vectors[i,2] * min_r_z + col_vectors[i,2] * (K-1-k)/float(K) * (max_r_z-min_r_z))
        seg = g.get_segment(i*K+k)
        if seg == 1: # show 'outside' class (which is inside the embryo)
            if x>=0 and y>=0 and z>=0 and \
               x<image.shape[2] and y<image.shape[1] and z<image.shape[0]:
                    imgout[z,y,x] = dotval_in_value
        else:
            if x>=0 and y>=0 and z>=0 and \
               x<image.shape[2] and y<image.shape[1] and z<image.shape[0]:
                    imgout[z,y,x] *= 1 # = dotval_out_value

In [61]:
volwin = volshow(imgout, stackUnits = [1.,1.,4.], raise_window=True)